In [17]:
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://en.wikipedia.org'

In [18]:
def get_partial_links(soup):
    """
    given beautiful soup messy output, extract the links (starting with '/wiki/')
    """
    print(f'getting partial links from raw list of {len(soup)}')
    partial_links = []
    for tag in soup:
        try:
            atag = tag.find('a')
            if atag is not None:
                link = atag['href']
                if link.startswith('/wiki') and ':' not in link:
                    partial_links.append(link)
        except KeyError:
            continue
    return partial_links

def get_members(infobox):
    """
    given an infobox, grab the members and past members
    """
    for child in infobox.parent.children:
        if child.text != 'Past members' and child.text != 'Members':
            return child.get_text(strip=True, separator=", ").split(',')

def scrape(url):
    """
    given a band wikipedia page, scrape the members and attributes
    and return their info in dataframe
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # get band name
    band = soup.find('h1', class_='firstHeading')

    # get current members
    members = []
    df = pd.DataFrame()

    infobox = soup.find(class_='infobox-label', string='Members')
    if infobox is not None:
        members = get_members(infobox)
        if members is not None:
            if len(members) > 1:
                df = pd.DataFrame({'member': members, 
                                   'attr': 'current',
                                   'band': band})
            else:
                df = pd.DataFrame({'member': members, 
                                   'attr': 'current',
                                   'band': band}, index=[0])

    # get past members
    infobox = soup.find(class_='infobox-label', string='Past members')
    if infobox is not None:
        past_members = get_members(infobox)
        if past_members is not None:
            if len(past_members) > 1:
                df = df.append(pd.DataFrame({'member': past_members, 
                                             'attr': 'past',
                                             'band': band}))
            else:
                df = df.append(pd.DataFrame({'member': past_members, 
                                             'attr': 'past',
                                             'band': band}, index=[0]))    

    try:
        df['years active'] = soup.find('th', class_='infobox-label', string='Years active').parent.find('td').text
    except:
        pass
    
    try:
        df['origin'] = soup.find('th', class_='infobox-label', string='Origin').parent.find('td').text
    except:
        pass

    return df

## Scrape early bands
A more manageable dataset

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_heavy_metal_bands'
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
table = soup.find_all('tbody')[1]

partial_links = [a['href'] for a in table.find_all('a')]
band_links = [link for link in partial_links if link.startswith('/wiki')]

In [20]:
df_list = []

for link in band_links:
    print(link)
    url = base_url + link
    df_list.append(scrape(url))

/wiki/220_Volt_(band)
/wiki/44_Magnum_(band)
/wiki/A_II_Z
/wiki/Accept_(band)
/wiki/Speed_metal
/wiki/Teutonic_thrash_metal
/wiki/AC/DC
/wiki/Aerosmith
/wiki/Alice_Cooper_(band)
/wiki/Angel_(American_band)
/wiki/%C3%81ngeles_del_Infierno
/wiki/Angel_Witch
/wiki/Anthem_(band)
/wiki/Anthrax_(American_band)
/wiki/Anvil_(band)
/wiki/Armageddon_(A%26M_band)
/wiki/List_of_Atomic_Mass_%26_Def_Leppard_members
/wiki/Atomic_Rooster
/wiki/Atomkraft
/wiki/Attila_(rock_band)
/wiki/Billy_Joel
/wiki/Axe_(band)
/wiki/Babe_Ruth_(band)
/wiki/Bang_(American_band)
/wiki/Barnabas_(band)
/wiki/Bar%C3%B3n_Rojo
/wiki/Battleaxe_(band)
/wiki/Australian_heavy_metal#History
/wiki/Beow%C3%BClf
/wiki/Birth_Control_(band)
/wiki/Bitch_(band)
/wiki/Metal_Blade_Records
/wiki/Black_Death_(American_band)
/wiki/Black_%27n_Blue
/wiki/Tommy_Thayer
/wiki/Kiss_(band)
/wiki/Black_Rose_(British_band)
/wiki/Black_Sabbath
/wiki/Ozzy_Osbourne
/wiki/Tony_Iommi
/wiki/Geezer_Butler
/wiki/Bill_Ward_(musician)
/wiki/Rainbow_(English_ba

In [21]:
early = pd.concat([item for item in df_list if type(item) != str])
print(len(early))
# early.to_csv('early_bands.csv', index=False)

2894


## Scrape bands from all sub genres
This takes a long time

In [46]:
## get list of other sub-genres
url = base_url + '/wiki/List_of_heavy_metal_bands'
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
genres_list = soup.find_all('ul')[0].find_all('li')

genre_links = get_partial_links(genres_list)
genre_links = [link for link in genre_links if link.startswith('/wiki/List_of')]

sub_genres = []
for genre_url in genre_links:
    url = base_url + genre_url
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    bands_table = soup.find('table', class_='wikitable')
    if bands_table is None:
        bands_list = soup.find_all('li')
    else:
        bands_list = bands_table.find_all('a')
    try:
        links = get_partial_links(bands_list)
        sub_genres.append(links)
    except:
        continue

getting partial links from raw list of 29


In [146]:
df_list = []
for genre, band_list in zip(genre_links, sub_genres):
    print(genre)
    for url in band_list:
        df = scrape(base_url + url)
        print(f'adding {len(df)} members for {url}')
        df['genre'] = genre
        df_list.append(df)
    time.sleep(3)

/wiki/List_of_alternative_metal_bands
/wiki/List_of_black_metal_bands
adding 0 members for /wiki/List_of_black_metal_bands,_0%E2%80%93K
adding 0 members for /wiki/List_of_black_metal_bands,_L%E2%80%93Z
adding 0 members for /wiki/List_of_black_metal_bands
adding 0 members for /wiki/List_of_black_metal_bands
adding 0 members for /wiki/Main_Page
/wiki/List_of_Christian_metal_bands
adding 12 members for /wiki/2Tm2,3
adding 4 members for /wiki/3rd_Root
adding 7 members for /wiki/7_Angels_7_Plagues
adding 9 members for /wiki/7_Horns_7_Eyes
adding 14 members for /wiki/12_Stones
adding 6 members for /wiki/38th_Parallel_(band)
adding 7 members for /wiki/Abated_Mass_of_Flesh
adding 10 members for /wiki/Ace_Augustine
adding 13 members for /wiki/Admonish_(band)
adding 6 members for /wiki/Advent_(band)
adding 8 members for /wiki/Aggelos_(band)
adding 15 members for /wiki/The_Agony_Scene
adding 9 members for /wiki/Aletheian
adding 10 members for /wiki/Alisa_(Russian_band)
adding 10 members for /wiki

adding 4 members for /wiki/HB_(band)
adding 8 members for /wiki/He_Is_Legend
adding 3 members for /wiki/Heaven%27s_Force
adding 16 members for /wiki/Here_I_Come_Falling
adding 15 members for /wiki/A_Hill_to_Die_Upon
adding 26 members for /wiki/Holy_Blood_(band)
adding 12 members for /wiki/Holy_Soldier
adding 4 members for /wiki/Hope_for_the_Dying
adding 12 members for /wiki/Hopesfall
adding 1 members for /wiki/Horde_(band)
adding 2 members for /wiki/Hortor
adding 9 members for /wiki/Hundredth_(band)
adding 13 members for /wiki/I,_the_Breather
adding 13 members for /wiki/I_Am_Terrified
adding 8 members for /wiki/Immortal_Souls
adding 4 members for /wiki/Impellitteri
adding 12 members for /wiki/Impending_Doom_(American_band)
adding 5 members for /wiki/Indwelling_(band)
adding 4 members for /wiki/Inevitable_End
adding 10 members for /wiki/Inhale_Exhale
adding 10 members for /wiki/Inked_in_Blood_(band)
adding 7 members for /wiki/In_the_Midst_of_Lions
adding 10 members for /wiki/Islander_(b

adding 12 members for /wiki/Stretch_Arm_Strong
adding 9 members for /wiki/Strongarm_(band)
adding 0 members for /wiki/Stryken
adding 10 members for /wiki/Stryper
adding 3 members for /wiki/Sympathy_(band)
adding 10 members for /wiki/Symphony_in_Peril
adding 6 members for /wiki/Symphony_of_Heaven
adding 7 members for /wiki/Taking_the_Head_of_Goliath
adding 6 members for /wiki/Tantrum_of_the_Muse
adding 0 members for /wiki/Tempest_(Christian_rock_band)
adding 6 members for /wiki/Temple_of_Blood
adding 12 members for /wiki/Temple_of_Perdition
adding 8 members for /wiki/Texas_In_July
adding 10 members for /wiki/Theocracy_(band)
adding 3 members for /wiki/Thieves_%26_Liars
adding 12 members for /wiki/This_Is_Hell_(band)
adding 17 members for /wiki/This_or_the_Apocalypse
adding 9 members for /wiki/Those_Who_Fear
adding 8 members for /wiki/Thousand_Foot_Krutch
adding 7 members for /wiki/A_Thousand_Times_Repent
adding 6 members for /wiki/Through_Solace
adding 2 members for /wiki/Tim%C5%8Dr%C4%

adding 0 members for /wiki/Hymnody_of_continental_Europe
adding 0 members for /wiki/Exclusive_psalmody
adding 0 members for /wiki/Jesus_music
adding 0 members for /wiki/Sacred_Harp
adding 0 members for /wiki/Shape_note
adding 0 members for /wiki/Spiritual_(music)
adding 0 members for /wiki/Contemporary_Christian_music
adding 0 members for /wiki/Contemporary_worship_music
adding 0 members for /wiki/Gospel_music
adding 0 members for /wiki/Neues_Geistliches_Lied
adding 0 members for /wiki/Mass_(music)
adding 0 members for /wiki/Christian_country_music
adding 0 members for /wiki/Christian_electronic_dance_music
adding 0 members for /wiki/Christian_hip_hop
adding 0 members for /wiki/Christian_R%26B
adding 0 members for /wiki/Christian_rock
adding 0 members for /wiki/Christian_alternative_rock
adding 0 members for /wiki/Christian_metal
adding 0 members for /wiki/Unblack_metal
adding 0 members for /wiki/Christian_punk
adding 0 members for /wiki/Christian_hardcore
adding 0 members for /wiki/Ch

adding 16 members for /wiki/Antestor
adding 1 members for /wiki/Ascending_King
adding 8 members for /wiki/Ashen_Mortality
adding 12 members for /wiki/Ashes_You_Leave
adding 5 members for /wiki/Ava_Inferi
adding 14 members for /wiki/Benea_Reach
adding 7 members for /wiki/Bilocate
adding 4 members for /wiki/Black_Debbath
adding 1 members for /wiki/Black_Sabbath_(band)
adding 9 members for /wiki/Bloody_Panda
adding 7 members for /wiki/Bongzilla
adding 4 members for /wiki/Boris_(band)
adding 3 members for /wiki/Brainoil_(band)
adding 3 members for /wiki/Burial_Chamber_Trio
adding 6 members for /wiki/Burning_Witch
adding 9 members for /wiki/Buzzov*en
adding 16 members for /wiki/Candlemass_(band)
adding 1 members for /wiki/Catacombs_(band)
adding 10 members for /wiki/Cathedral_(band)
adding 11 members for /wiki/Celtic_Frost
adding 3 members for /wiki/Celestiial
adding 17 members for /wiki/Church_of_Misery
adding 9 members for /wiki/Cirith_Ungol_(band)
adding 10 members for /wiki/Confessor_(b

...

adding 0 members for /wiki/List_of_nu_metal_bands
adding 0 members for /wiki/List_of_nu_metal_bands
adding 0 members for /wiki/Main_Page
/wiki/List_of_new_wave_of_British_heavy_metal_bands
adding 5 members for /wiki/Alkatrazz
adding 31 members for /wiki/Angel_Witch
adding 3 members for /wiki/Atomkraft
adding 0 members for /wiki/A_II_Z
adding 5 members for /wiki/Avenger_(British_band)
adding 0 members for /wiki/Baby_Tuckoo
adding 12 members for /wiki/Battleaxe_(band)
adding 0 members for /wiki/Paul_Di%27Anno#Battlezone_(1985–89,_1998)
adding 4 members for /wiki/Black_Rose_(UK_band)
adding 7 members for /wiki/Bleak_House_(band)
adding 5 members for /wiki/Blitzkrieg_(heavy_metal_band)
adding 0 members for /wiki/Bronz
adding 0 members for /wiki/Charlie_%E2%80%99Ungry
adding 6 members for /wiki/Chateaux_(band)
adding 13 members for /wiki/Chrome_Molly
adding 6 members for /wiki/Cloven_Hoof_(band)
adding 18 members for /wiki/Dedringer
adding 8 members for /wiki/Def_Leppard
adding 6 members fo

adding 0 members for /wiki/African_heavy_metal
adding 0 members for /wiki/Argentine_heavy_metal
adding 0 members for /wiki/Australian_thrash_metal
adding 0 members for /wiki/Australian_heavy_metal
adding 0 members for /wiki/Bangladeshi_heavy_metal
adding 0 members for /wiki/Bay_Area_thrash_metal
adding 0 members for /wiki/Brazilian_thrash_metal
adding 0 members for /wiki/Chinese_heavy_metal
adding 0 members for /wiki/Florida_death_metal
adding 0 members for /wiki/Hungarian_metal
adding 0 members for /wiki/Indigenous_metal_music
adding 0 members for /wiki/Japanese_metal
adding 11 members for /wiki/Les_L%C3%A9gions_Noires
adding 0 members for /wiki/New_wave_of_American_heavy_metal
adding 0 members for /wiki/New_wave_of_British_heavy_metal
adding 0 members for /wiki/Early_Norwegian_black_metal_scene
adding 0 members for /wiki/Palm_Desert_Scene
adding 0 members for /wiki/Swedish_death_metal
adding 0 members for /wiki/Teutonic_thrash_metal
adding 0 members for /wiki/Ukrainian_metal
adding 0

adding 8 members for /wiki/Caligula%27s_Horse
adding 10 members for /wiki/Candiria
adding 11 members for /wiki/Cattle_Decapitation
adding 5 members for /wiki/Carbonized
adding 6 members for /wiki/Circus_Maximus_(Norwegian_band)
adding 0 members for /wiki/Confessor
adding 3 members for /wiki/Communic
adding 8 members for /wiki/Coheed_and_Cambria
adding 5 members for /wiki/Conception_(band)
adding 8 members for /wiki/Coroner_(band)
adding 10 members for /wiki/Crimson_Glory
adding 2 members for /wiki/Cronian
adding 14 members for /wiki/Cynic_(band)
adding 14 members for /wiki/Dark_Suns
adding 8 members for /wiki/Dead_Brain_Cells
adding 5 members for /wiki/Death_(metal_band)
adding 8 members for /wiki/The_Dillinger_Escape_Plan
adding 11 members for /wiki/Deus_Invictus_(band)
adding 5 members for /wiki/Dir_En_Grey
adding 7 members for /wiki/Distorted_Harmony
adding 10 members for /wiki/Dream_Theater
adding 4 members for /wiki/Dysrhythmia_(band)
adding 7 members for /wiki/Echoes_of_Eternity


/wiki/List_of_symphonic_black_metal_bands
/wiki/List_of_symphonic_metal_bands
adding 1 members for /wiki/Abigail_Williams_(band)
adding 5 members for /wiki/Ad_Infinitum_(symphonic_metal_band)
adding 14 members for /wiki/Adagio_(band)
adding 10 members for /wiki/After_Forever
adding 10 members for /wiki/Alia_Tempora
adding 1 members for /wiki/Almora_(band)
adding 12 members for /wiki/Amberian_Dawn
adding 8 members for /wiki/Ancient_Bards
adding 9 members for /wiki/...And_Oceans
adding 15 members for /wiki/Angra_(band)
adding 5 members for /wiki/Angtoria
adding 7 members for /wiki/Anorexia_Nervosa_(band)
adding 16 members for /wiki/Antestor
adding 6 members for /wiki/Apocalyptica
adding 6 members for /wiki/Arcturus_(band)
adding 0 members for /wiki/Arthemesia
adding 9 members for /wiki/Arven_(band)
adding 8 members for /wiki/Atargatis_(band)
adding 14 members for /wiki/Autumn_(Dutch_band)
adding 6 members for /wiki/Avantasia
adding 2 members for /wiki/Ayreon
adding 11 members for /wiki/B

adding 2 members for /wiki/Welicoruss
adding 6 members for /wiki/Whyzdom
adding 19 members for /wiki/Winds_of_Plague
adding 6 members for /wiki/Wintersun
adding 17 members for /wiki/Within_Temptation
adding 4 members for /wiki/Xandria
adding 18 members for /wiki/X_Japan
adding 8 members for /wiki/Xerath
adding 0 members for /wiki/Neoclassical_metal
adding 0 members for /wiki/Power_metal
adding 0 members for /wiki/List_of_symphonic_metal_bands
adding 0 members for /wiki/List_of_symphonic_metal_bands
adding 0 members for /wiki/Main_Page
/wiki/List_of_thrash_metal_bands
/wiki/List_of_Viking_metal_bands


In [148]:
results = pd.concat(df_list)
results = results.drop_duplicates(['member', 'band'])
# results.to_csv('subgenres.csv', index=False)

## Save everything together

In [160]:
early['genre'] = 'early'
results = subgenres.append(early)
results.to_csv('all_data.csv', index=False)